In [1]:
import os
import sys
import pyxrt
import contextlib
from neuroring import NeuroRingHost, NeuroRingKernel
import nest
import network
import numpy as np
from network_params import net_dict
from sim_params import sim_dict
from stimulus_params import stim_dict
from utils_binding import Options  # Ensure this module provides .index and .bitstreamFile

net = network.Network(sim_dict, net_dict, stim_dict)
net.create()
net.connect()

print(net.pops)

XRT build version: 2.18.179
Build hash: 3ade2e671e5ab463400813fc2846c57edf82bb10
Build date: 2024-11-05 13:57:47
Git branch: 2024.2
PID: 597730
UID: 1031
[Wed Aug 20 11:57:25 2025 GMT]
HOST: cyberdeck
EXE: /usr/bin/orted
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
XRT build version: 2.18.179
Build hash: 3ade2e671e5ab463400813fc2846c57edf82bb10
Build date: 2024-11-05 13:57:47
Git branch: 2024.2
PID: 597649
UID: 1031
[Wed Aug 20 11:57:26 2025 GMT]
HOST: 


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.8.0
 Built: Jan 27 2025 08:18:03

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



/home/miahafiz/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning:Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
/home/miahafiz/NeuroRing/host_py/network.py:210: UserWarning:DC input created to compensate missing Poisson input.



Data will be written to: /home/miahafiz/NeuroRing/host_py/data/
  Directory already existed. Old data will be overwritten.

Neuron numbers are scaled by a factor of 0.010.
Indegrees are scaled by a factor of 0.010.
  Weights and DC input are adjusted to compensate.

RNG seed: 55
Total number of virtual processes: 4
Creating neuronal populations.

Aug 20 13:57:27 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.
Creating recording devices.
  Creating spike recorders.
Creating DC generators for external stimulation.
Connecting neuronal populations recurrently.
Connecting DC generators.

Aug 20 13:57:28 NodeManager::prepare_nodes [Info]: 
    Preparing 835 nodes for simulation.
[NodeCollection(metadata=None, model=iaf_psc_exp, size=207, first=1, last=207), NodeCollection(metadata=None, model=iaf_psc_exp, size=58, first=208, last=265), NodeCollection(metadata=None, model=iaf_psc_exp, size=219, first=266, last=484), NodeCollection(metadata=None, mode

In [2]:
host = NeuroRingHost(net, 1, 1, "/home/miahafiz/NeuroRing/krnl_neuroring_hw.xclbin")
host.initialize_devices()

Extracting synapse information...
Initialized device 0 with XCLBIN UUID: ed4759a8-983c-e7d6-879c-703a5be42805
threshold: -50.0
membrane_potential: -65.0
dcstim_amp: 209.11304111722728
{'simulation_time': 1, 'threshold': 3259498496, 'membrane_potential': 3263299584, 'amount_of_cores': 1, 'neuron_start': 1, 'neuron_total': 771, 'dcstim_start': 0, 'dcstim_total': 1000, 'dcstim_amp': 1129389296, 'device': None, 'xclbin': None, 'uuid': None, 'kernel_name': None, 'kernel': None, 'synapseListHandle': None, 'header_words': 7680000, 'header_bytes': 30720000, 'tail_words_capacity': 7710000, 'tail_bytes_capacity': 30840000, 'bo_size': 61560000}
Initialized kernel NeuroRing:{NeuroRing_0} and AxonLoader:{AxonLoader_0} on device <pyxrt.device object at 0x7f6c5afb2270>
Allocated BO of 61560000 bytes (header 30720000, tail 30840000)


In [3]:
host.save_array_to_csv(host.packed_list_per_fpga[0][0], "Synapse_list.csv")

In [ ]:
import time

start_time = time.time()
host.run_kernels(1000)
host.wait_for_kernels(1000)
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

In [ ]:
array = host.kernels_per_fpga[0][0].get_spike_recorder_array(10000)
print(array)
print(len(array))
print("--------------------------------")

for i in range(len(array)):
    if array[i] != 0:
        print(f"{i} {array[i]}")

In [ ]:
print(len(host.kernels_per_fpga[0][0].synapse_list_data_final))
print(host.packed_list_per_fpga[0][0][0:100])
print(host.kernels_per_fpga[0][0].dcstim_amp)

In [ ]:
host.kernels_per_fpga[0][0].run_kernel(host.packed_list_per_fpga[0][0], 2)
host.kernels_per_fpga[0][1].run_kernel(host.packed_list_per_fpga[0][1], 2)

In [ ]:
host.kernels_per_fpga[0][0].wait_for_axon()
host.kernels_per_fpga[0][1].wait_for_axon()
host.kernels_per_fpga[0][2].wait_for_axon()
host.kernels_per_fpga[0][3].wait_for_axon()


In [ ]:
host.kernels_per_fpga[0][1].wait_for_axon()

In [ ]:
host.kernels_per_fpga[0][0].wait_for_neuroring()
host.kernels_per_fpga[0][1].wait_for_neuroring()
host.kernels_per_fpga[0][2].wait_for_neuroring()
host.kernels_per_fpga[0][3].wait_for_neuroring()

In [ ]:
host.kernels_per_fpga[0][0].wait_for_neuroring()

In [ ]:
host.kernels_per_fpga[0][0].wait_for_kernel()
print(host.kernels[0].spikeRecorder_array)


In [ ]:
for i in range(len(host.kernels[0].spikeRecorder_array)):
    if host.kernels[0].spikeRecorder_array[i] > 0:
        print(i)

In [ ]:
print(host.kernels[0].spikeRecorder_array)


In [ ]:
print(host.packed_list_per_fpga[0][0][0:110])
print(host.packed_list_per_fpga[0][0][10000:10010])

In [ ]:
print(net.DC_amp)
print(np.average(net.DC_amp))

In [ ]:
print(host.kernels_per_fpga[0][1].kernel.group_id(0))
print(host.kernels_per_fpga[0][1].kernel.group_id(1))


In [ ]:
print(host.packed_list_per_fpga[0][0][0:10])
print(host.packed_list_per_fpga[0][0][0+(1*10000):10+(1*10000)])
print(host.packed_list_per_fpga[0][0][0+(2*10000):10+(2*10000)])
print(host.packed_list_per_fpga[0][0][0+(3*10000):10+(3*10000)])
print(host.packed_list_per_fpga[0][0][0+(4*10000):10+(4*10000)])
print(host.packed_list_per_fpga[0][0][0+(5*10000):10+(5*10000)])
print(host.packed_list_per_fpga[0][0][0+(6*10000):10+(6*10000)])
print(host.packed_list_per_fpga[0][0][0+(7*10000):10+(7*10000)])
print(host.packed_list_per_fpga[0][0][0+(8*10000):10+(8*10000)])
print(host.packed_list_per_fpga[0][0][0+(9*10000):10+(9*10000)])
print(host.packed_list_per_fpga[0][0][0+(10*10000):10+(10*10000)])
print(host.packed_list_per_fpga[0][0][0+(11*10000):10+(11*10000)])
print(host.packed_list_per_fpga[0][0][0+(12*10000):10+(12*10000)])
print(host.packed_list_per_fpga[0][0][0+(13*10000):10+(13*10000)])
print(host.packed_list_per_fpga[0][0][0+(14*10000):10+(14*10000)])
print(host.packed_list_per_fpga[0][0][0+(15*10000):10+(15*10000)])


In [ ]:

print(net.pops[1])